<a href="https://colab.research.google.com/github/mnmahir/FYProject-PCGIPI/blob/main/System_web_app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Library**

In [ ]:
!pip install anvil-uplink
import anvil.server
import anvil.media
# Google Drive
from google.colab import drive
# General
import math
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import keras
!pip install image_slicer
import os
import glob
import image_slicer
# Keras Library
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator, load_img
# Others
import os

from google.colab import drive

#**Load Model**

In [ ]:
drive.mount('/content/gdrive')
model_directory = '/content/gdrive/MyDrive/FYP Stuff/Model/' #@param {type: "string"}
model_name = "DenseNet201_model.h5" #@param ["DenseNet201_200AK4_model.h5", "ResNet50V2_200AK4_model.h5", "ResNet101V2_200AK4_model.h5", "VGG16_200AK4_model.h5", "VGG19_200AK4_model.h5", "Xception_200AK4_model.h5", "DenseNet201_model.h5"]
model = tf.keras.models.load_model(model_directory + model_name)

#**Main**

In [ ]:
###########################
# Initialization
###########################
IMAGE_CLASSES = ['NORMAL','GRADE 1','GRADE 2','GRADE 3']
IMAGE_WIDTH,IMAGE_HEIGHT = (224,224)
sliced_img_path = '/content/sliced_img'
try: 
  os.mkdir(sliced_img_path) # For before calling slice
except:
  print("Folder already created")

#####################################
# Slice image / remove sliced images
#####################################
#To be called before making prediction
def slice_image(ori_img, srow = 6, scol = 8): #later add row and col
  os.chdir(sliced_img_path)

  ori_img.save('ori.png')
  if scol > 1 or srow > 1:
    image_slicer.slice('ori.png', row = srow, col = scol)
    os.remove('ori.png') #Delete original image
  os.chdir('/content')

# To be called after done with prediction and calculation
def remove_sliced_images():
  files = glob.glob(sliced_img_path+'/*')  
  for f in files:
      os.remove(f)                      # Delete everything in folder

# Get slicing parameter (200px)
def get_slice_param(img):
  img_width, img_height = img.size
  swidth = int(img_width/200) #To get 200px
  sheight = int(img_height/200)
  owidth = int(img_width/swidth)
  oheight = int(img_height/sheight)
  return swidth, sheight, owidth, oheight

#####################################
# Make prediction (Will predict all images in the "sliced_img" folder)
#####################################
def get_predictions():
  predictions = []
  for i,img in enumerate(sorted(os.listdir(sliced_img_path))):
    img = os.path.join(sliced_img_path, img)
    img = image.load_img(img, target_size=(IMAGE_WIDTH,IMAGE_HEIGHT))
    arr = image.img_to_array(img)
    arr = np.expand_dims(arr, axis=0)
    arr /= 255
    predictions.append(model.predict(arr))
  pred_labels = np.argmax(predictions, axis=-1)
  return predictions, pred_labels

############################
# Save all prediction patches
############################
def save_predicted_patches(predictions, pred_labels, srow = 6, scol = 8, conf_thrs = 0.8):
  imeg_freq = len(pred_labels)
  disp_row = srow

  plt.figure(1, figsize = (scol*2, srow*2))
  for j,img in enumerate(sorted(os.listdir(sliced_img_path))):
    img = image.load_img(os.path.join(sliced_img_path,img), target_size=(IMAGE_WIDTH,IMAGE_HEIGHT))
    plt.subplot(srow, scol, j+1)

    if predictions[j][0,pred_labels[j][0]] < conf_thrs:
      #title = "Low Confidence"
      #plt.title(title, y = 0, fontsize = 15, fontweight = 'bold')
      plt.axis('off')

    elif pred_labels[j][0] == 0:
      title = str(IMAGE_CLASSES[pred_labels[j][0]]) + '\n' + str(round(predictions[j][0,pred_labels[j][0]],4))
      plt.title(title, y = 0, fontsize = 15, fontweight = 'bold', color = 'lime')
      for axis in ['top','bottom','left','right']:
        plt.gca().spines[axis].set_linewidth(7)
        plt.gca().spines[axis].set_color("lime")

    elif pred_labels[j][0] == 1:
      title = str(IMAGE_CLASSES[pred_labels[j][0]]) + '\n' + str(round(predictions[j][0,pred_labels[j][0]],4))
      plt.title(title, y = 0, fontsize = 15, fontweight = 'bold', color = 'orange')
      for axis in ['top','bottom','left','right']:
        plt.gca().spines[axis].set_linewidth(7)
        plt.gca().spines[axis].set_color("orange")
    elif pred_labels[j][0] == 2:
      title = str(IMAGE_CLASSES[pred_labels[j][0]]) + '\n' + str(round(predictions[j][0,pred_labels[j][0]],4))
      plt.title(title, y = 0, fontsize = 15, fontweight = 'bold', color = 'magenta')
      for axis in ['top','bottom','left','right']:
        plt.gca().spines[axis].set_linewidth(7)
        plt.gca().spines[axis].set_color("magenta")

    elif pred_labels[j][0] == 3:
      title = str(IMAGE_CLASSES[pred_labels[j][0]]) + '\n' + str(round(predictions[j][0,pred_labels[j][0]],4))
      plt.title(title, y = 0, fontsize = 15, fontweight = 'bold', color = 'red')
      for axis in ['top','bottom','left','right']:
        plt.gca().spines[axis].set_linewidth(7)
        plt.gca().spines[axis].set_color("red")

    else:
      pass
    #plt.axis('off')
    #plt.gca().spines[axis].set_zorder(0)
    #title = str(IMAGE_CLASSES[pred_labels[j][0]]) + '\n' + str(round(predictions[j][0,pred_labels[j][0]],2))
    #plt.title(title, y = 0.75, loc = 'left')
    plt.tick_params(bottom = False, labelbottom = False, left = False, labelleft = False)
    plt.imshow(img)
  plt.tight_layout(pad = 0)
  plt.savefig('predicted_patches.png', bbox_inches='tight', pad_inches = 0)
  plt.close()
############################
# Calculate frequency of class prediction
############################
def get_pred_freq(predictions, pred_labels, conf_thrs = 0.8):
  freq_class = [0,0,0,0]
  for j in range(len(pred_labels)):
    if predictions[j][0,pred_labels[j][0]] >= conf_thrs:
      freq_class[pred_labels[j][0]] += 1
  max_class = freq_class.index(max(freq_class))
  return freq_class, IMAGE_CLASSES[max_class]


############################
# Anvil Uplink
############################
uplink_key = "L3PK5MYQVJ4ZK5FTWKPZYKUD-FBM5VHPXV3N7HNS4"#@param {type: "string"}
anvil.server.connect(uplink_key) 
@anvil.server.callable
def predict_click(img_file, conf_threshold,  swidth, sheight):
  global image_received, score, model_name
  with anvil.media.TempFile(img_file) as filename:
    img = load_img(filename)
  image_received = img
  slice_image(img, srow = sheight, scol = swidth)
  predictions, pred_labels = get_predictions()
  save_predicted_patches(predictions, pred_labels, srow = sheight, scol = swidth, conf_thrs = conf_threshold)
  class_freq, max_class = get_pred_freq(predictions, pred_labels, conf_thrs = conf_threshold)
  remove_sliced_images()

  send_image = anvil.media.from_file('predicted_patches.png', '/content/') #To convert into anvil media object (filename, path)
  return class_freq, max_class , send_image, model_name

anvil.server.wait_forever()

# Debug Code

In [ ]:
def get_pred_freq(predictions, pred_labels, conf_thrs = 0.8):
  freq_class = [0,0,0,0]
  for j in range(len(pred_labels)):
    if predictions[j][0,pred_labels[j][0]] >= conf_thrs:
      freq_class[pred_labels[j][0]] += 1
  max_class = freq_class.index(max(freq_class))
  return freq_class, IMAGE_CLASSES[max_class]

In [ ]:
test,test2 = get_pred_freq(predictions, pred_labels)

In [ ]:
print(test,test2)

In [ ]:
save_predicted_patches(predictions, pred_labels, srow = 6, scol = 8, conf_thrs = 0.5)

In [ ]:
print(image_received)

In [ ]:
slice_image(image_received, srow = 4, scol = 4)
predictions, pred_labels = get_predictions()
save_predicted_patches(predictions, pred_labels, srow = 4, scol = 4)

In [ ]:
srow = 6
scol = 8
os.chdir(sliced_img_path)
ori_img = image_received
ori_img.save('ori.png')
if scol > 1 or srow > 1:
  sliced_img = image_slicer.slice('ori.png', row = srow, col = scol, save = False)
  os.remove('ori.png') #Delete original image
os.chdir('/content')

In [ ]:
print(sliced_img)

In [ ]:
remove_sliced_images()

In [ ]:
#testlist = np.ndarray.tolist(predictions)
print(predictions[0][0,1]) #[index][fixed,class]

In [ ]:
print(predictions)

In [ ]:
print(pred_labels)

In [ ]:
# Run after click predict in anvil
print(image_received)
plt.imshow(image_received)
plt.savefig('test.png')
print(score)
a = get_label(score)
#print (IMAGE_CLASSES[a[0]])
slice_image(image_received)
print('Size: ',image_received.size)
